In [8]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.

# BUILDING A DISTANCE HELPER EXCEL VERSION FOR PYTHON

from IPython.display import display, Markdown, clear_output, HTML, Javascript, display_html
import subprocess
import sys
display(Markdown(""" ## Checking required packages:
#### if any are missing, installing them now..... """))
!{sys.executable} -m pip install --no-cache-dir -r requirements_SP.txt
clear_output()


import ipysheet
from ipysheet import column,sheet,to_dataframe,row
import ipywidgets as widgets
import pandas as pd
import geopy
import collections
import ipywidgets as widgets
from geopy.geocoders import Nominatim
import numpy as np
from itertools import combinations, permutations, product
from math import radians, cos, sin, asin, sqrt
from ast import literal_eval as make_tuple

import os
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
clear_output()
#display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+1)'))


HTML('''<script>{
    $('div.input').hide();
    }''')

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+1)'))
HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    The raw code for this IPython notebook is by default hidden for easier reading toggle on/off the raw code by clicking <a href="javascript:code_toggle()">here</a>.''')




<IPython.core.display.Javascript object>

In [9]:
display(Markdown("#  Distant Helper"))
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


#  Distant Helper

<IPython.core.display.Javascript object>

In [10]:
style = {'description_width': '185px'}
layout = {'width': '365px'}

get_to_next = widgets.Button(description='Go',button_style='primary')
howmanyBLDG = widgets.Text(description='Number of buildings',style=style)
def go_next_line(click):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

clear_output()
display(Markdown("#### <font color='Green'> 1. Enter the total number of Buildings you have. </font>"))
display(howmanyBLDG)
display(get_to_next)
get_to_next.on_click(go_next_line)


#### <font color='Green'> 1. Enter the total number of Buildings you have. </font>

Text(value='', description='Number of buildings', style=DescriptionStyle(description_width='185px'))

Button(button_style='primary', description='Go', style=ButtonStyle())

<IPython.core.display.Javascript object>

In [11]:

styletext = {'description_width': '200px'}
howmanyFLRS = widgets.Text(description='Max number of floors',style=styletext)
# style_sheet = {'description_width': '1200px'}
# sheetnameBLDGS = ipysheet.sheet(rows=int(howmanyBLDG.value), columns=3, column_headers=True, row_headers=False,layout=layout,style=style_sheet,row_resizing=True,column_resizing=True)

# sheetnameBLDGS.column_headers=['Campus Name','Building','Address']
# exec("for i in range("+str(howmanyBLDG.value)+"):\n\trow(i, ['', '', ''])")
# # column(0, ['', '', ''])
# create an Empty DataFrame 
# object With column names only 
dfskeleton_BLDG = pd.DataFrame(np.zeros([int(howmanyBLDG.value), 3])*np.nan)
dfskeleton_BLDG1 = dfskeleton_BLDG.replace(np.nan, '', regex=True)
dfskeleton_BLDG1.columns = ['Campus Name','Building','Address']
dfskeleton_BLDG1.reset_index(drop=True, inplace=True)

sheetnameBLDGS = ipysheet.from_dataframe(dfskeleton_BLDG1)

clear_output()
display(Markdown("#### <font color='Green'> 2. Enter your campus/building information below. </font>"))
display(Markdown("##### <font color='Gray'> Address is needed to fetch latitude and longitude corrdinates. </font>"))
maxflrs_mkdwn = widgets.Output()
with maxflrs_mkdwn:
    display(Markdown("#### <font color='Green'> 3. Find what is the maximum number of floors and enter it here. </font>"))
display(widgets.VBox([sheetnameBLDGS,widgets.VBox([maxflrs_mkdwn,howmanyFLRS]),get_to_next]))
    


#### <font color='Green'> 2. Enter your campus/building information below. </font>

##### <font color='Gray'> Address is needed to fetch latitude and longitude corrdinates. </font>

In [ ]:
import time

display(Markdown(""" #### Fetching Latitude and Longitude coordinates... """))

dfbuildingADDRESS = to_dataframe(sheetnameBLDGS) 

addresscol = 'Address'
global maxfloors
maxfloors = int(howmanyFLRS.value)

# declare an empty list to store 
# latitude and longitude of values  
# of column 
longitude = [] 
latitude = [] 



def findlatlong_1(addresscol): 
# try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
        time.sleep(1) #to add delay in case of large DFs  
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="get_lat_long") 
#         location = geolocator.geocode(col)
#         coordinates = location.latitude, location.longitude  
        return geolocator.geocode(addresscol)
      
    except GeocoderTimedOut: 
          
        return findlatlong_1(addresscol) 
    
    
# each value from column 
# will be fetched and sent to 
# function find_geocode 
for i in (dfbuildingADDRESS[addresscol]): 

    if findlatlong_1(i) != None: 

        loc = findlatlong_1(i) 

        # coordinates returned from  
        # function is stored into 
        # two separate list 
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 

    # if coordinate for a city not 
    # found, insert "NaN" indicating  
    # missing value  
    else: 
        latitude.append(np.nan) 
        longitude.append(np.nan) 
clear_output()
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
 #now add these columns to dataframe 
dfbuildingSTG = dfbuildingADDRESS.copy()
dfbuildingSTG["Latitude"] = latitude 
dfbuildingSTG["Longitude"] = longitude 
#create coordinates column from both
dfbuildingSTG['Coordinates'] = dfbuildingSTG['Latitude'].map('{:,.6f}'.format) +','+ dfbuildingSTG['Longitude'].map('{:,.6f}'.format)

#grab final output df
dfbuildingSTG.drop(['Latitude','Longitude','Address'],axis=1,inplace=True)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
columnsFLR = ['Number of Floors']
columnsFLR = columnsFLR + ['Floor Number '+str(i) for i in range(1,maxfloors+1)]

dfbuildingSTG2 = dfbuildingSTG.copy()
dfbuildingSTG3 = pd.concat([dfbuildingSTG2,pd.DataFrame(None,columns=columnsFLR)],sort=False)
dfbuildingSTG4 = dfbuildingSTG3.replace(np.nan, '', regex=True)
dfbuildingSTG4['Building ID'] = dfbuildingSTG4['Campus Name']+'-'+dfbuildingSTG4['Building']

bid_col_name="Building ID"
first_col_bldg = dfbuildingSTG4.pop(bid_col_name)

dfbuildingSTG4.insert(0, bid_col_name, first_col_bldg)
sheetnameBLDGFLRS = ipysheet.from_dataframe(dfbuildingSTG4)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
# clear_output()
display(Markdown("#### <font color='Green'> 4. Enter your floor numbers below. </font>"))
display(widgets.VBox([sheetnameBLDGFLRS,get_to_next]))

In [ ]:
maxzones_mkdwn = widgets.Output()
with maxzones_mkdwn:
    display(Markdown("#### <font color='Green'> 5. Find what is the maximum number of zones you have and enter it here. </font>"))

howmanyQDRTS = widgets.Text(description='Max number of zones',style=styletext)
display(widgets.VBox([maxzones_mkdwn,howmanyQDRTS,get_to_next]))

In [ ]:
dfbuildingsFINAL = to_dataframe(sheetnameBLDGFLRS)
global maxquadrants
maxquadrants = int(howmanyQDRTS.value)

flrqdrt_style = {'width:50px'}

columnsQDRT = ['Floor Name','Number of Zones']
columnsQDRT = columnsQDRT + ['Zone Coordinates '+str(i) for i in range(1,maxquadrants+1)]
df_selectneededCOLS = dfbuildingsFINAL[['Campus Name','Building', 'Number of Floors']].copy()

dfrepeatedbyFLRS = pd.DataFrame(np.repeat(df_selectneededCOLS.values, 
                                           df_selectneededCOLS['Number of Floors'].replace(0,1).tolist(), 
                                           axis=0),
                                 columns=df_selectneededCOLS.columns)

dffloorSTG1 = pd.concat([dfrepeatedbyFLRS,pd.DataFrame(None,columns=columnsQDRT)],sort=False)
dffloorSTG2 = dffloorSTG1.replace(np.nan, '', regex=True)

sheetnameFLRS = ipysheet.from_dataframe(dffloorSTG2)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
clear_output()
display(Markdown("#### <font color='Green'> 6. Enter your floor names and zone details below. </font>"))
display(Markdown("### <font color='Gray'> <b> Game of Zones</b> </font>"))
display(Markdown("#### <font color='black'> <b> Please use the following syntax for zone coordinates. E.G. (1,1)</b> </font>"))
display(widgets.VBox([sheetnameFLRS,get_to_next]))


In [ ]:
df_quadrants_selected = to_dataframe(sheetnameFLRS)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
clear_output()

In [ ]:
#dffloorsFINAL = to_dataframe(sheetnameFLRS)
dffloorsFINAL = df_quadrants_selected.copy()
# dffloorsFINAL= dffloorsFINAL.fillna(value=np.nan)
dffloorsFINAL = dffloorsFINAL.replace([None], [''], regex=True)
dffloorsFINAL['Floor ID'] = dffloorsFINAL['Campus Name']+'-'+dffloorsFINAL['Building']+'-'+dffloorsFINAL['Floor Name']

fid_col_name="Floor ID"
first_col_flr = dffloorsFINAL.pop(fid_col_name)

dffloorsFINAL.insert(0, fid_col_name, first_col_flr)
dffloorsFINAL = dffloorsFINAL.rename(columns={'Campus Name': 'Campus'})


dffloorsFINAL_melted = pd.melt(dffloorsFINAL, id_vars=['Floor ID','Campus','Building','Number of Floors','Floor Name','Number of Zones'], var_name='Zone Id', value_name='Zone Coordinates')
dffloorsFINAL_melted['Space Name'] = ""

df_rename_ZONEQDRNTS = dffloorsFINAL_melted[['Floor ID','Number of Zones','Zone Coordinates','Space Name']]

df_rename_ZONEQDRNTS = df_rename_ZONEQDRNTS.sort_values('Floor ID')
df_rename_ZONEQDRNTS['Zone Coordinates'].replace('', np.nan, inplace=True)
df_rename_ZONEQDRNTS.dropna(subset=['Zone Coordinates'], inplace=True)

sheetnameZONES_NAMES = ipysheet.from_dataframe(df_rename_ZONEQDRNTS)

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
display(Markdown("#### <font color='Green'> 7. Enter your space names for each coordinate. </font>"))
display(Markdown("#### <font color='black'> <b> These 'friendly' names should match your Floor values in your Space Capacity file. </b> </font>"))

display(widgets.VBox([sheetnameZONES_NAMES,get_to_next]))

In [ ]:
df_zones_STG = to_dataframe(sheetnameZONES_NAMES)

# check if space names 
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
"""
https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points

"""
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    #r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    r = 3956
    return c * r


##############################
#Load Files ** Start Here **
############################

cwd = os.getcwd()
# tab_nest_001 = widgets.Accordion()
# style = {'description_width': '185px'}
# layout = {'width': '365px'}

# go_run_full = widgets.Button(description='Go',button_style = 'primary')

# def go_to_nextblock(btn):
#         display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+3)'))


# input_path_001 = FileChooser(cwd)
# tab_nest_001.children = [input_path_001]

# tab_nest_001.set_title(0,'Load file and name output file')
# go_run_full.on_click(go_to_nextblock) 


# display(widgets.VBox([tab_nest_001,go_run_full]))


dfbuildingsDIST = dfbuildingsFINAL.copy()
dfbuildingsDIST.index = dfbuildingsDIST.index.astype(int) #use astype to convert to int
dfbuildingsDIST.columns = [s.strip() for s in dfbuildingsDIST.columns]
num_bldgs = len(dfbuildingsDIST)
dfbuildingsDIST[['lat','long']] = dfbuildingsDIST["Coordinates"].str.split(",",expand=True) 
dfbuildingsDIST['lat'] = dfbuildingsDIST['lat'].astype(float)
dfbuildingsDIST['long'] = dfbuildingsDIST['long'].astype(float)

for i in range(num_bldgs):
    bldg_id = dfbuildingsDIST.loc[i,"Building ID"]
    bldg_bmcRaw = dfbuildingsDIST.loc[i,"Coordinates"]
    blat = dfbuildingsDIST.loc[i,"lat"]
    blong = dfbuildingsDIST.loc[i,"long"]
    #print(bldg_id, "coordinates found...", blat, blong)

#print("calculating building distance...")

bld_ids = list(range(len(dfbuildingsDIST)))
bldg_edges = list(product(bld_ids, bld_ids))
#bldg_edges = list(permutations(bld_ids, 2))
bldg_distDF = pd.DataFrame(bldg_edges, columns = ["from_idx", "to_idx"])

dfbuildingsDIST2 = dfbuildingsDIST[["Building ID", "lat", "long"]].copy()

bldg_distDF = pd.merge(bldg_distDF, dfbuildingsDIST2, left_on = "from_idx", right_index=True, how = 'left')
bldg_distDF = pd.merge(bldg_distDF, dfbuildingsDIST2, left_on = "to_idx", right_index=True, how = 'left', suffixes= ['_From', '_To'])
for i in range(len(bldg_distDF)):
    lat_from = bldg_distDF.loc[i,"lat_From"]
    long_from = bldg_distDF.loc[i,"long_From"]
    lat_to = bldg_distDF.loc[i,"lat_To"]
    long_to = bldg_distDF.loc[i,"long_To"]
    hav_dist = haversine(lat_from, long_from, lat_to, long_to)
    bldg_distDF.loc[i,"dist"] = hav_dist
    
    
#print("Everything good so far... ")
#print("Calculated " + str(len(bldg_distDF))+ " distances across building and campus using Bing Map coordinates and Haversine formula")


################################
## Floor Distances
#################################

#print("calculating floor to floor distances...")

#create floors
def create_floor_distances(floor_rel_dist, avg_dist):
    rel_floor_idxs = list(range(len(floor_rel_dist)))
    res = []
    floor_rel_dist_avg = [x * avg_dist for x in floor_rel_dist]
    for origin in floor_rel_dist_avg:
        calc_dist_row = [abs(x - origin) for x in floor_rel_dist_avg]
        res.append(calc_dist_row.copy())
    return res, rel_floor_idxs  

def create_floor_adj_list(K, key_start = 0):
    if K == 0: return [], None
    if K == 1: return [key_start], {key_start: []}
    ids = [key_start+i for i in list(range(K))]
    ids_set = set(ids)
    d = {}
    for current in ids:
        d[current] = sorted(list(ids_set - set([current])))
    return ids, d

bldg_input = dfbuildingsFINAL.copy()
bldg_input.index = bldg_input.index.astype(int) #use astype to convert to int
num_bldgs = len(bldg_input)
floors_per_bldg = {}
dist_per_floors = {}
floor_labels_per_bldg = {}
cols_b_tab = list(bldg_input.columns)
f_start_idx = cols_b_tab.index("Floor Number 1")
#print("loading floor relative distances....")
for i in range(num_bldgs):
    bldg_id = dfbuildingsDIST.loc[i,"Building ID"]
    num_floors = int(dfbuildingsDIST.loc[i,"Number of Floors"])
    #removing optional column to input average distance as per @july 20, 2020 excel input file validation review meeting
    #avg_dist = df1.loc[i,"Average Distance per Floor"] 
    avg_dist = 2
    floor_rel_dist = list(dfbuildingsDIST.iloc[i,f_start_idx: f_start_idx + num_floors])
    floor_rel_dist = [int(i) for i in floor_rel_dist] 
    #print(bldg_id, "found", num_floors, "floors rel dist: ", floor_rel_dist)
    floor_distances, floors_idx = create_floor_distances(floor_rel_dist, avg_dist)
    key_start = 0
    floor_labels = [key_start+i for i in floors_idx]  
    dist_per_floors[bldg_id]= floor_distances.copy()
    floor_labels_per_bldg[bldg_id] = floor_labels.copy()
    
    fi, fn = create_floor_adj_list(num_floors)
    if fn:
        floors_per_bldg[bldg_id] = fn.copy()
    else:
        print("wait....something went wrong....Buildings dataframe (dfbuildingsFINAL)... row:", i)

#print("number of buildings with floor dist:", len(dist_per_floors), "of", len(floors_per_bldg) )


##########################
## Quadrant Coordinates
##########################

#print("loading quadrant coordinates... from floors tab... ")
df100 = dffloorsFINAL.copy()
#df100 = dffloorsFINAL.copy()
df100.index = df100.index.astype(int) #use astype to convert to int
df100.columns = [s.strip() for s in df100.columns]
num_floors = len(df100)
quad_coord_per_floor = {}

cols = list(df100.columns)
start_idx = cols.index('Zone Coordinates 1')
#print("start of coordinates columns detected... column idx:", start_idx)
#print("validating coordinate columns vs number of quadrants... ")

for i in range(num_floors):
    floor_id = df100.loc[i,"Floor ID"]
    num_quads = int(df100.loc[i,"Number of Zones"])
    coord_row = []
    for j in range(num_quads):
        temp = df100.iloc[i, start_idx+j]
        coord_row.append(temp)
    if num_quads != len(coord_row):
        print(floor_id, "number of zones do not match")
        print("Excel Floor tab ERROR: Please enter one coordinate per each zone")
    else:
        #print(floor_id,"found quadrant coordinates...", coord_row)
        quad_coord_per_floor[floor_id] = coord_row.copy() 

        
    ##########################
## Quadrant Distances
##########################
#print("calculating quadrant to quadrant distances...")

#create quadrants
def create_quad_distances(coords, avg_dist):
    coord_idxs = list(range(len(coords)))
    coords_arr = [ np.array(x) for x in coords] 
    res = []
    for origin in coords_arr:
        coord_dist_row = [np.linalg.norm(coord2-origin) for coord2 in coords_arr]
        res.append(coord_dist_row.copy())
    return res, coord_idxs  

def create_quad_adj_list(K, key_start = "a"):
    if K == 0: return [], None
    if K == 1: return [key_start], {key_start: ""}
    ids = [chr(ord(key_start)+i) for i in list(range(K))]
    ids_set = set(ids)
    d = {}
    for current_quad in ids:
        d[current_quad] = "".join(sorted(list(ids_set - set(current_quad))))
    return ids, d


quads_dist_per_floor = {}
quad_idxs_map_per_floor = {}
quad_labels_per_floor = {} 
for i in range(num_floors):
    floor_id = df100.loc[i,"Floor ID"]
    #TO DO. Average distance per quadrant small/medium/large
    #avg_dist_q = df100.loc[i,"Average Distance Per Quadrant"] 
    avg_dist_q = 1
    coords = quad_coord_per_floor[floor_id]
    coords = [make_tuple(s) for s in coords]
    coords = [(float(x),float(y)) for x,y in coords]
    quad_distances, quads_idx = create_quad_distances(coords, avg_dist_q)
    key_start_q = 'a'
    quad_labels = [chr(ord(key_start_q)+i) for i in quads_idx]
    quad_idxs_map = {q: i for q,i in zip(quad_labels, quads_idx)}
    quads_dist_per_floor[floor_id]= quad_distances.copy()
    quad_labels_per_floor[floor_id] = quad_labels.copy()
    quad_idxs_map_per_floor[floor_id] = quad_idxs_map.copy() 

#print("number of floors with quads dist", len(quads_dist_per_floor))

#############################
#  Quadrant IDs
#############################

quadrant_ids = []
for floor_id, quad_labels in quad_labels_per_floor.items():
    campus_name, bldg_name, floor_name = floor_id.split("-")  
    bldg_id = campus_name + "-" + bldg_name 
    for q in quad_labels:
        space_row = {}
        qid = floor_id+"-"+q
        space_row["campus"] = campus_name
        space_row["building"] = bldg_name
        space_row["floor"] = floor_name
        space_row["quad"] = q
        quad_idx_map = quad_idxs_map_per_floor[floor_id]
        space_row["quad_idx"] = quad_idx_map[q]
        space_row["building_id"] = bldg_id
        space_row["floor_id"] = floor_id
        space_row["quadrant_id"] = qid
        quadrant_ids.append(space_row.copy())        

#print("number of spaces loaded... ", len(quadrant_ids))
####################################
#  Create Campus and Building Matrix
####################################

#sort idxs
quadrant_idsDF = pd.DataFrame(quadrant_ids)
quadrant_idsDF = quadrant_idsDF.sort_values(by = ["quadrant_id"]).reset_index(drop = True)
quadrant_idsDF = quadrant_idsDF.reset_index()

#new campus and buildings matrix (filled with zeros)
N = len(quadrant_ids)
new_building_expanded_matrix = np.zeros((N,N))

#print("New building matrix created... expanding to quad size... shape:", new_building_expanded_matrix.shape )
#print("adding building distances as per Bing Map coordintes...")

proxyDF = quadrant_idsDF
bldg_T = new_building_expanded_matrix
for i in range(len(bldg_distDF)):
    from_bldg = bldg_distDF.loc[i,"Building ID_From"]
    to_bldg = bldg_distDF.loc[i,"Building ID_To"]
    bldg_dist = bldg_distDF.loc[i,"dist"]
    from_idxs = list(proxyDF.loc[proxyDF["building_id"]==from_bldg,"index"]) 
    to_idxs = list(proxyDF.loc[proxyDF["building_id"]==to_bldg,"index"]) 
    for i in from_idxs:
        for j in to_idxs:
            bldg_T[i, j] = bldg_dist

#print("building matrix filled...")

#output for excel
campus_bldg_DF = pd.DataFrame(bldg_T)
campus_bldg_DF.index = list(quadrant_idsDF["quadrant_id"])
campus_bldg_DF = campus_bldg_DF.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
campus_bldg_DF.columns = new_cols

####################################
#  Create Floors Distance Matrix
####################################

#new floors matrix (filled with zeros)
new_floors_expanded_matrix = np.zeros((N,N))

#print("New floors matrix created... expanding to quad size... shape:", new_floors_expanded_matrix.shape )
#print("adding floors distances as per buildings tab...")

proxyDF = quadrant_idsDF
floor_T = new_floors_expanded_matrix
for bldg_id, floor_distances in dist_per_floors.items():
    floor_labels = sorted(set(list(proxyDF.loc[proxyDF['building_id'] == bldg_id, "floor"])))
    floor_labels_map = { i:s for i,s in enumerate(floor_labels)}
    for rel_floor_idx1, floor_dist_row in enumerate(floor_distances):                        
        from_floor = floor_labels_map[rel_floor_idx1]
        for rel_floor_idx2, floor_dist in enumerate(floor_dist_row): 
            to_floor = floor_labels_map[rel_floor_idx2]
            from_idxs = list(proxyDF.loc[(proxyDF["building_id"] == bldg_id) & (proxyDF["floor"]==from_floor),"index"]) 
            to_idxs = list(proxyDF.loc[(proxyDF["building_id"] == bldg_id) & (proxyDF["floor"]==to_floor),"index"]) 
            for i in from_idxs:
                for j in to_idxs:
                    floor_T[i, j] = floor_dist

#print("floor matrix filled...")

#output for excel
floors_DF = pd.DataFrame(floor_T)
floors_DF.index = list(quadrant_idsDF["quadrant_id"])
floors_DF = floors_DF.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
floors_DF.columns = new_cols

####################################
# "REPMAT" Floor Vectors - Horizontally
####################################
#print("translating floor distances horizontally... ")

proxyDF = quadrant_idsDF
floor_T2 = np.zeros((N,N))
for bldg_id, floor_distances in dist_per_floors.items():
    row_idxs = list(proxyDF.loc[(proxyDF["building_id"] == bldg_id),"index"]) 
    col_idxs = row_idxs.copy()
    cv, rv = np.meshgrid(row_idxs, col_idxs)
    temp = floor_T[rv, cv]
    floor_dist = temp[:,[0]]  #first column
    floor_vector = floor_dist
    #print("REPMAT horizontally", bldg_id, "num rows:", len(floor_vector))
    min_idx, max_idx = min(col_idxs), max(col_idxs)
    #load left 
    floor_T2[row_idxs, 0:min_idx] = floor_vector
    #load right
    floor_T2[row_idxs, max_idx+1:] = floor_vector

#output for excel
floors_DF2 = pd.DataFrame(floor_T2)
floors_DF2.index = list(quadrant_idsDF["quadrant_id"])
floors_DF2 = floors_DF2.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
floors_DF2.columns = new_cols

#print("Floor matrix HORIZONTAL translation matrix filled...")

####################################
# "REPMAT" Floor Vectors - Vertically
####################################
#print("translating floor distances VERTICALLY... ")

proxyDF = quadrant_idsDF
floor_T3 = np.zeros((N,N))
for bldg_id, floor_distances in dist_per_floors.items():
    row_idxs = list(proxyDF.loc[(proxyDF["building_id"] == bldg_id),"index"]) 
    col_idxs = row_idxs.copy()
    cv, rv = np.meshgrid(row_idxs, col_idxs)
    temp = floor_T[rv, cv]
    floor_dist = temp[[0],:]  #first row
    floor_vector = floor_dist
    #print("REPMAT vertically", bldg_id, "num rows:", len(floor_vector))
   
    min_idx, max_idx = min(col_idxs), max(col_idxs)
   
    #load left 
    floor_T3[0:min_idx, col_idxs] = floor_vector
    #load right
    floor_T3[max_idx+1:, col_idxs] = floor_vector
   
   

#output for excel
floors_DF3 = pd.DataFrame(floor_T3)
floors_DF3.index = list(quadrant_idsDF["quadrant_id"])
floors_DF3 = floors_DF3.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
floors_DF3.columns = new_cols

#print("Floor matrix VERTICAL translation matrix filled...")

###################################################################
# Add "Translate" Floor Vectors - Vertical + Horizontal + Diagonal
###################################################################

floor_T4 = floor_T + floor_T2 + floor_T3

#output for excel
floors_DF4 = pd.DataFrame(floor_T4)
floors_DF4.index = list(quadrant_idsDF["quadrant_id"])
floors_DF4 = floors_DF4.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
floors_DF4.columns = new_cols


####################################
#  Create Quadrants Distance Matrix
####################################

#new quadrants matrix (filled with zeros)
new_quad_expanded_matrix = np.zeros((N,N))

#print("New quad matrix created... expanding to quad size... shape:", new_quad_expanded_matrix.shape )
#print("adding quad distances as per coordinates in floors tab...")

proxyDF = quadrant_idsDF
quad_T = new_quad_expanded_matrix
for floor_id, quad_distances in quads_dist_per_floor.items():
    quad_labels = quad_labels_per_floor[floor_id]
    quad_labels_map = { i:s for i,s in enumerate(quad_labels)}
    for rel_quad_idx1, quad_dist_row in enumerate(quad_distances):                        
        from_quad = quad_labels_map[rel_quad_idx1]
        for rel_quad_idx2, quad_dist in enumerate(quad_dist_row): 
            to_quad = quad_labels_map[rel_quad_idx2]
            from_idxs = list(proxyDF.loc[(proxyDF["floor_id"] == floor_id) & (proxyDF["quad"]==from_quad),"index"]) 
            to_idxs = list(proxyDF.loc[(proxyDF["floor_id"] == floor_id) & (proxyDF["quad"]==to_quad),"index"]) 
            for i in from_idxs:
                for j in to_idxs:
                    quad_T[i, j] = quad_dist

#print("quad matrix filled...")

#output for excel

quads_DF = pd.DataFrame(quad_T)
quads_DF.index = list(quadrant_idsDF["quadrant_id"])
quads_DF = quads_DF.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
quads_DF.columns = new_cols

####################################
# "REPMAT" Quad Vectors - Horizontally
####################################
#print("translating quad distances horizontally... ")

proxyDF = quadrant_idsDF
quad_T2 = np.zeros((N,N))
for floor_id, quad_distances in quads_dist_per_floor.items():
    quad_dist = quad_distances[0]
    quad_vector = np.array(quad_dist).reshape(len(quad_dist),1)
    #print("REPMAT horizontally", floor_id, "num rows:", len(quad_vector))
    row_idxs = list(proxyDF.loc[(proxyDF["floor_id"] == floor_id),"index"]) 
    col_idxs = row_idxs.copy()
    min_idx, max_idx = min(col_idxs), max(col_idxs)
    #load left 
    quad_T2[row_idxs, 0:min_idx] = quad_vector
    #load right
    quad_T2[row_idxs, max_idx+1:] = quad_vector

#output for excel
quads_DF2 = pd.DataFrame(quad_T2)
quads_DF2.index = list(quadrant_idsDF["quadrant_id"])
quads_DF2 = quads_DF2.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
quads_DF2.columns = new_cols

#print("quad matrix HORIZONTAL translation matrix filled...")

####################################
# "REPMAT" Quad Vectors - Vertical
####################################
#print("translating quads distances vertically... ")
proxyDF = quadrant_idsDF
quad_T3 = np.zeros((N,N))
for floor_id, quad_distances in quads_dist_per_floor.items():
    quad_dist = quad_distances[0]
    quad_vector = np.array(quad_dist).reshape(1, len(quad_dist))
    #print("REPMAT vertically", floor_id, "num rows:", len(quad_vector))
    row_idxs = list(proxyDF.loc[(proxyDF["floor_id"] == floor_id),"index"]) 
    col_idxs = row_idxs.copy()
    min_idx, max_idx = min(row_idxs), max(row_idxs)
    #load up
    quad_T3[0:min_idx, col_idxs] = quad_vector
    #load right
    quad_T3[max_idx+1:, col_idxs] = quad_vector

#output for excel
quads_DF3 = pd.DataFrame(quad_T3)
quads_DF3.index = list(quadrant_idsDF["quadrant_id"])
quads_DF3 = quads_DF3.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
quads_DF3.columns = new_cols

#print("quad matrix VERTICAL translation matrix filled...")

##############################################################
# Add "Translate" Quad Vectors - Vertical + Horizontal + Diagonal
##############################################################

quad_T4 = quad_T + quad_T2 + quad_T3

#output for excel
quads_DF4 = pd.DataFrame(quad_T4)
quads_DF4.index = list(quadrant_idsDF["quadrant_id"])
quads_DF4 = quads_DF4.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
quads_DF4.columns = new_cols


##########>  TO DOs. 
# 
#               1) expand floor and quads vector (and/or affine transformation for final distance across floors quads)
#
#               2) calibrate quads coordinate (e.g. sqrt(2))
#
#               3) calibrate buildings, floors quads order of magnitude - bing map vs quads coordinates vs floor rel dist proportions according to CS needs (e.g. "zoom in" into quad vs "zoom in" floors, etc.) 
# 
###############################>>>

bldg_boost = 1000
floor_boost = 20
quad_boost = 1

final_T = bldg_T*bldg_boost + floor_T4*floor_boost + quad_T4*quad_boost + bldg_T* 5280

#output for excel
final_DF = pd.DataFrame(final_T)
final_DF.index = list(quadrant_idsDF["quadrant_id"])
final_DF = final_DF.reset_index()
quad_ids = list(quadrant_idsDF["quadrant_id"])
new_cols = ["quadrant_id"] + quad_ids
final_DF.columns = new_cols

quad_ids_keys = quad_ids
quad_ids_values = list(df_zones_STG['Space Name'])

# using zip() 
# to convert lists to dictionary 
quad_rename_dict  = dict(zip(quad_ids_keys, quad_ids_values)) 

final_DF2 = final_DF.copy()
# Remap the values of the dataframe 
final_DF2['quadrant_id']= final_DF2['quadrant_id'].map(quad_rename_dict) 
final_DF2 = final_DF2.rename(columns=quad_rename_dict)


final_DF2.to_csv('distance.csv',index = False)



#w2f.save()
#clear_output()

showdshp = widgets.Button(description ='Show details')
hidedshp = widgets.Button(description ='Hide details')
dshp_mkdn = widgets.Output()
with dshp_mkdn:
    display(Markdown('<b>Distance logging</b>'))
display(dshp_mkdn)
display(showdshp)

def click_on_hide_showdshp_details(click):
    clear_output()
    display(dshp_mkdn)
    display(showdshp)

def click_on_show_showdshp_details(click):
    clear_output()
    display(dshp_mkdn)
    display(hidedshp)
#     print("Loading space catalog from... ")
#     print("folder:", input_path)
#     print("file:", input_file)

#     print("--------------------------------------------------------")
#     for tab_name in xl1.sheet_names:
#         temp = xl1.parse(tab_name).fillna(0)
#         print("reading excel tab...", tab_name," shape... ", temp.shape)
#         tabs.append(temp.copy())

    print("loading buildings...")

    for i in range(num_bldgs):
        bldg_id = dfbuildingsDIST.loc[i,"Building ID"]
#         bldg_id.index = bldg_id.index.astype(int) #use astype to convert to int
        bldg_bmcRaw = dfbuildingsDIST.loc[i,"Coordinates"]
        blat = dfbuildingsDIST.loc[i,"lat"]
        blong = dfbuildingsDIST.loc[i,"long"]
        print(bldg_id, "coordinates found...", blat, blong)

    print("--------------------------------------------------------")

    print("calculating building distance...")

    print("Everything good so far... ")
    print("Calculated " + str(len(bldg_distDF))+ " distances across building and campus using Coordinates and Haversine formula")

    print("--------------------------------------------------------")

    print("calculating floor to floor distances...")
    print("loading floor relative distances....")


    for i in range(num_bldgs):
        bldg_id = dfbuildingsDIST.loc[i,"Building ID"]
        num_floors = int(dfbuildingsDIST.loc[i,"Number of Floors"])
        #removing optional column to input average distance as per @july 20, 2020 excel input file validation review meeting
        #avg_dist = df1.loc[i,"Average Distance per Floor"] 
        avg_dist = 2
        floor_rel_dist = list(dfbuildingsDIST.iloc[i,f_start_idx: f_start_idx + num_floors])
        floor_rel_dist = [int(i) for i in floor_rel_dist] 
        print(bldg_id, "found", num_floors, "floors rel dist: ", floor_rel_dist)



    if fn:
            floors_per_bldg[bldg_id] = fn.copy()
    else:
        print("wait....something went wrong....first tab... row:", i)


    print("number of buildings with floor dist:", len(dist_per_floors), "of", len(floors_per_bldg) )


    print("--------------------------------------------------------")

    print("loading quadrant coordinates... from floors tab... ")

    print("start of coordinates columns detected... column idx:", start_idx)
    print("validating coordinate columns vs number of zones... ")

    for i in range(num_floors):
        floor_id = df100.loc[i,"Floor ID"]
        num_quads = int(df100.loc[i,"Number of Zones"])
        coord_row = []
        for j in range(int(num_quads)):
            temp = df100.iloc[i, start_idx+j]
            coord_row.append(temp)
        if num_quads != len(coord_row):
            print(floor_id, "number of zones do not match")
            print("Excel Floor tab ERROR: Please enter one coordinate per each zone")
        else:
            print(floor_id,"found zone coordinates...", coord_row)
            quad_coord_per_floor[floor_id] = coord_row.copy() 

    print("--------------------------------------------------------")

    print("calculating zone to zone distances...")
    print("number of floors with quads dist", len(quads_dist_per_floor))
    print("number of spaces loaded... ", len(quadrant_ids))




    print("--------------------------------------------------------")

    print("New building matrix created... expanding to quad size... shape:", new_building_expanded_matrix.shape )
    print("adding building distances as per Coordinates...")


    print("building matrix filled...")


    print("--------------------------------------------------------")
    print("New floors matrix created... expanding to quad size... shape:", new_floors_expanded_matrix.shape )
    print("adding floors distances as per buildings tab...")
    print("floor matrix filled...")

    print("--------------------------------------------------------")
    print("translating floor distances horizontally... ")
    print("Floor matrix HORIZONTAL translation matrix filled...")
    print("--------------------------------------------------------")
    for bldg_id, floor_distances in dist_per_floors.items():
        row_idxs = list(proxyDF.loc[(proxyDF["building_id"] == bldg_id),"index"]) 
        col_idxs = row_idxs.copy()
        cv, rv = np.meshgrid(row_idxs, col_idxs)
        temp = floor_T[rv, cv]
        floor_dist = temp[[0],:]  #first row
        floor_vector = floor_dist
        print("REPMAT vertically", bldg_id, "num rows:", len(floor_vector))


    print("Floor matrix VERTICAL translation matrix filled...")


    print("--------------------------------------------------------")

    print("New quad matrix created... expanding to quad size... shape:", new_quad_expanded_matrix.shape )
    print("adding quad distances as per coordinates in floors tab...")

    print("quad matrix filled...")


    print("--------------------------------------------------------")
    print("translating quad distances horizontally... ")

    for floor_id, quad_distances in quads_dist_per_floor.items():
        quad_dist = quad_distances[0]
        quad_vector = np.array(quad_dist).reshape(len(quad_dist),1)
        print("REPMAT horizontally", floor_id, "num rows:", len(quad_vector))


    print("quad matrix HORIZONTAL translation matrix filled...")

    print("--------------------------------------------------------")

    print("translating quads distances vertically... ")
    for floor_id, quad_distances in quads_dist_per_floor.items():
        quad_dist = quad_distances[0]
        quad_vector = np.array(quad_dist).reshape(1, len(quad_dist))
        print("REPMAT vertically", floor_id, "num rows:", len(quad_vector))

    print("quad matrix VERTICAL translation matrix filled...")


    print("--------------------------------------------------------")



showdshp.on_click(click_on_show_showdshp_details)
hidedshp.on_click(click_on_hide_showdshp_details)

# print(os.getcwd())
# print(base_path)
# print(base_path + out_filename)

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
savedshpfinsh = '<div class="alert alert-block alert-success"><b> Done: </b> Your distant output file <b>(distance.csv)</b> has successfully saved back to location of this notebook.('+ os.getcwd() +')</div>'
display(Markdown(savedshpfinsh))
style = {'description_width': '300px'}
layout = {'width': '225px'}
startValidationsNB = widgets.Button(description ='Open Validations UI Notebook',style=style,layout=layout,button_style='info')
display(startValidationsNB)

def go_to_nextblock(btn):
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

startValidationsNB.on_click(go_to_nextblock)           
   

In [ ]:
%%javascript
var name_of_the_notebook = 'WPP.Input-File-Validations_PIPE-002_v2.ipynb'
var url = window.location.href.split('/')
var newurl = url[0] + '//'
for (var i = 1; i < url.length - 1; i++) {
    console.log(url[i], newurl)
    newurl += url[i] + '/'
}
newurl += name_of_the_notebook
window.open(newurl)